In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import numpy as np
from datetime import date, timedelta

In [2]:
def cell_to_num(char):
    char = char.upper()
    if char.isalpha():
        num = ord(char) - 64
    else:
        num = 0
    return num
print(cell_to_num('a'))

1


In [3]:
"""
# Windows file system
first_file = 'C:/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = 'C:/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'
"""
# linux file system
first_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'

wb_check = load_workbook(first_file)

/home/pankaj/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
def data_cleaning(path):
    wb = load_workbook(path)
    ws = wb.active
    
    #Unmerge cell
    ws.unmerge_cells('O1:P1')
    ws.unmerge_cells('Q1:R1')
    ws.unmerge_cells('S1:T1')
    ws.unmerge_cells('U1:V1')
    if path[-7] == '1':
        #renaming issue_sale value with dates
        ws['S1'] = str(date.today() - timedelta(days=4))
        ws['T1'] = str(date.today() - timedelta(days=3))
    else:
        ws['S1'] = str(date.today() - timedelta(days=2))    
        ws['T1'] = str(date.today() - timedelta(days=1))
    
    print(ws['S1'].value)
    print(ws['T1'].value)    
    
    #Delete column U:V
    ws.delete_cols(cell_to_num('U'), cell_to_num('V')-cell_to_num('U')+1)
    #Delete column O:R
    ws.delete_cols(cell_to_num('O'), cell_to_num('R')-cell_to_num('O')+1)
    #Delete column K:M
    ws.delete_cols(cell_to_num('K'), cell_to_num('M')-cell_to_num('K')+1)
    #Delete column B:F
    ws.delete_cols(cell_to_num('B'), cell_to_num('F')-cell_to_num('B')+1)
    #Delete second row
    ws.delete_rows(2)
    wb.save(path)

In [5]:
def create_pivot(path):
    #reading excel file using pandas
    old_df = pd.read_excel(path)
    current_wb = load_workbook(path)
    ws = current_wb.active
    
    #filter out the 'NA' value in the product column
    df = old_df[
        (old_df['Product'] == 'XP') |
        (old_df['Product'] == 'HS') |
        (old_df['Product'] == 'MS')
    ]
    
    df.loc[df['Product'] == 'XP', 'Product'] = 'MS'
    
    pvt_all = pd.pivot_table(df,
                            index=["SA"],
                            columns=['Product'],
                            values=[ws['H1'].value, ws['G1'].value],
                            aggfunc=np.sum)
    return pvt_all

In [6]:
data_cleaning(first_file)
data_cleaning(second_file)

2020-08-16
2020-08-17
2020-08-14
2020-08-15


In [7]:
pvt = pd.concat([create_pivot(first_file), create_pivot(second_file)], axis=1)
print(pvt)

/home/pankaj/.local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                     2020-08-16           2020-08-17           2020-08-14  \
Product                      HS        MS         HS        MS         HS   
SA                                                                          
Fuels- Bargarh(R)      32835.87  25309.81   34876.52  40595.59   51458.34   
Fuels-Bhawanipatna(R   39575.28  31448.45   40162.58  46243.38   56846.22   
Fuels-JEYPORE(R)       50165.91  38469.76   66110.27  60491.78   96573.48   
Fuels-Jharsuguda(R)    72217.43  34428.30   91649.51  37756.00  156818.81   
Fuels-ROURKELA(R)     258624.98  38925.13  240717.77  48156.52  358783.29   
Fuels-SAMBALPUR (R)    65449.13  19537.97   61286.04  29178.34   87110.36   
Keonjhar I RSA         81805.33  17257.64   75716.21  20835.06  109009.41   
Keonjhar II RSA         3594.88   6057.07    6778.15   8040.71    8665.76   

                               2020-08-15            
Product                     MS         HS        MS  
SA                                          

In [8]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    pvt.to_excel(writer, 'pivot sheet', index=True)

In [9]:
df = pd.read_excel(first_file, sheet_name='pivot sheet')
print(df.columns.values)

['Unnamed: 0' '2020-08-16' 'Unnamed: 2' '2020-08-17' 'Unnamed: 4'
 '2020-08-14' 'Unnamed: 6' '2020-08-15' 'Unnamed: 8']


In [10]:
cols = df.columns.values
order = [0, 3, 4, 1, 2, 7, 8, 5, 6]
cols = [cols[i] for i in order]
print(cols)

['Unnamed: 0', '2020-08-17', 'Unnamed: 4', '2020-08-16', 'Unnamed: 2', '2020-08-15', 'Unnamed: 8', '2020-08-14', 'Unnamed: 6']


In [11]:
df = df[cols]
print(df)

             Unnamed: 0 2020-08-17 Unnamed: 4 2020-08-16 Unnamed: 2  \
0               Product         HS         MS         HS         MS   
1                    SA        NaN        NaN        NaN        NaN   
2     Fuels- Bargarh(R)    34876.5    40595.6    32835.9    25309.8   
3  Fuels-Bhawanipatna(R    40162.6    46243.4    39575.3    31448.5   
4      Fuels-JEYPORE(R)    66110.3    60491.8    50165.9    38469.8   
5   Fuels-Jharsuguda(R)    91649.5      37756    72217.4    34428.3   
6     Fuels-ROURKELA(R)     240718    48156.5     258625    38925.1   
7   Fuels-SAMBALPUR (R)      61286    29178.3    65449.1      19538   
8        Keonjhar I RSA    75716.2    20835.1    81805.3    17257.6   
9       Keonjhar II RSA    6778.15    8040.71    3594.88    6057.07   

  2020-08-15 Unnamed: 8 2020-08-14 Unnamed: 6  
0         HS         MS         HS         MS  
1        NaN        NaN        NaN        NaN  
2    31916.3    26186.9    51458.3    53666.5  
3    41181.6    36227.6   

In [12]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    df.to_excel(writer, 'summary', index=False)

In [13]:
wb = load_workbook(first_file)
ws = wb['summary']
ws.delete_rows(3)
check = 0
for value in ws.iter_rows(min_row=1, max_row=1):
    for cell in value:
        if check % 2 == 0:
            cell.value = ""
        check += 1
        print(cell.value)
# print(df.columns.values)
print(ws['A3'].value)


2020-08-17

2020-08-16

2020-08-15

2020-08-14

Fuels- Bargarh(R)


In [14]:
"""
data = 'ABCDEFGHI'
for char in data:
    val = char + '1'
    print(ws[val].value)

print(ws.cell(row=1, column=2).value)

#Iterate through columns of the first row
for row in ws.iter_rows(min_row=1, max_row=1):
    for cell in row:
        print(cell.value)
"""

"\ndata = 'ABCDEFGHI'\nfor char in data:\n    val = char + '1'\n    print(ws[val].value)\n\nprint(ws.cell(row=1, column=2).value)\n\n#Iterate through columns of the first row\nfor row in ws.iter_rows(min_row=1, max_row=1):\n    for cell in row:\n        print(cell.value)\n"

In [15]:
from exchangelib import Account, Message, Credentials, HTMLBody
from exchangelib import Configuration, DELEGATE
import os

In [16]:
outlook_user = os.environ.get('OUTLOOK_USER')
outlook_password = os.environ.get('OUTLOOK_PASS')
outlook_server = os.environ.get('OUTLOOK_SERVER')
outlook_email = os.environ.get('OUTLOOK_EMAIL')

print(os.getenv('OUTLOOK_USER'))
print(os.getenv('OUTLOOK_PASS'))

ioc\00504802
Toptal@22


In [17]:
credentials = Credentials(username=outlook_user,
                         password=outlook_password
                         )
config = Configuration(server=outlook_server,
                      credentials=credentials)
account = Account(primary_smtp_address=outlook_email,
                 config=config,
                 autodiscover=False,
                 access_type=DELEGATE)

In [18]:
def create_html(ws):
    html = """
    <html><body><h3>Nozzle Sales Report:</h3>
    <table border=1>
    """
    # entering header data in html
    for whole_row in ws.iter_rows(min_row=1, max_row=2):
        html += "<tr>"
        for row in whole_row:
            html += f'<th>{row.value}</th>'
        html += "</tr>"
    # entering table data in html
    for whole_row in ws.iter_rows(min_row=3):
        html += "<tr>"
        for row in whole_row:
            if isinstance(row.value, float):
                html += f'<td>{round(row.value, 2)}</td>'
                print(round(row.value, 2))
            else:
                html += f'<td>{row.value}</td>'
        html += "</tr>"
    html += """
    </table>
    <p>
    <br>
    With Regards<br>
    Pankaj Barnwal<br>
    </p>
    </body>
    </html>
    """
    return html

In [19]:
msg = Message(
    account=account,
    subject="Nozzle Sales Report - Sambalpur DO",
    body=HTMLBody(create_html(ws)),
    to_recipients=['barnwalp@indianoil.in']
)

msg.send_and_save()

34876.52
40595.59
32835.87
25309.81
31916.28
26186.89
51458.34
53666.5
40162.58
46243.38
39575.28
31448.45
41181.62
36227.61
56846.22
44636.13
66110.27
60491.78
50165.91
38469.76
58347.37
37726.6
96573.48
57550.12
91649.51
37756.0
72217.43
34428.3
109397.97
22622.13
156818.81
47669.08
240717.77
48156.52
258624.98
38925.13
156148.62
14221.47
358783.29
72009.28
61286.04
29178.34
65449.13
19537.97
55817.3
18547.76
87110.36
39798.29
75716.21
20835.06
81805.33
17257.64
59621.21
12507.04
109009.41
25916.1
6778.15
8040.71
3594.88
6057.07
4921.45
4074.54
8665.76
9613.78
